In [50]:
%reload_ext lab_black

import pandas as pd
import numpy as np
from collections import Counter
import pandas as pd
import lightgbm as lgb
from sklearn.datasets import load_breast_cancer, load_boston, load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, roc_auc_score, precision_score

pd.options.display.max_columns = 999

In [51]:
final_df = pd.read_csv("../00_data/final_data_Apr10.csv")

In [62]:
final_df.columns

Index(['Unnamed: 0', 'viewCount', 'likeCount', 'dislikeCount',
       'num_days_since_pub', 'viewcounts_weighted', 'like_count_weighted',
       'dislike_count_weighted', 'like_dislike_ratio',
       'weighted_like_dislike_ratio',
       'like_dislike_ratio_by_viewcounts_weighted',
       'videoTitleSentimentScore_vader_pos',
       'videoTitleSentimentScore_vader_com', 'captionSentimentScore_vader_pos',
       'captionSentimentScore_vader_neg', 'captionSentimentScore_vader_com',
       'falir_videoTitle_score',
       'caption_list_split_by_nvidia_10words_flairscores',
       'caption_list_split_by_nvidia_20words_flairscores',
       'caption_list_split_by_nvidia_35words_flairscores',
       'caption_list_split_by_nvidia_10words_vaderscores',
       'caption_list_split_by_nvidia_20words_vaderscores',
       'caption_list_split_by_nvidia_35words_vaderscores', 'msft', 'amd',
       'ndaq', 'intel', 'qualcomm', 'apple', 'pred_24', 'pred_48', 'pred_72',
       'percent_change_24', 'percen

In [58]:
y = final_df.iloc[:, -1]
le = LabelEncoder()
le.fit(y)
y = le.transform(y)
print(list(le.classes_))
print(le.transform(["buy", "hold", "sell"]))

['buy', 'hold', 'sell']
[0 1 2]


In [53]:
final_df.action_96.unique()

array(['buy', 'hold', 'sell'], dtype=object)

In [54]:
def display_scores(scores):
    print(
        "Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(
            scores, np.mean(scores), np.std(scores)
        )
    )


def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


def algorithm_pipeline(
    X_train_data,
    X_test_data,
    y_train_data,
    y_test_data,
    model,
    param_grid,
    cv=10,
    scoring_fit="accuracy",
    do_probabilities=False,
):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv,
        n_jobs=-1,
        scoring=scoring_fit,
        verbose=2,
    )
    fitted_model = gs.fit(X_train_data, y_train_data)

    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)

    return fitted_model, pred

In [74]:
final_df.drop(
    [
        "Unnamed: 0",
        "msft",
        "amd",
        "ndaq",
        "intel",
        "qualcomm",
        "apple",
        "pred_24",
        "pred_48",
        "pred_72",
        "action_96",
        "percent_change_24",
        "percent_change_48",
        "percent_change_72",
    ],
    axis=1,
)

,viewCount,likeCount,dislikeCount,num_days_since_pub,viewcounts_weighted,like_count_weighted,dislike_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted,videoTitleSentimentScore_vader_pos,videoTitleSentimentScore_vader_com,captionSentimentScore_vader_pos,captionSentimentScore_vader_neg,captionSentimentScore_vader_com,falir_videoTitle_score,caption_list_split_by_nvidia_10words_flairscores,caption_list_split_by_nvidia_20words_flairscores,caption_list_split_by_nvidia_35words_flairscores,caption_list_split_by_nvidia_10words_vaderscores,caption_list_split_by_nvidia_20words_vaderscores,caption_list_split_by_nvidia_35words_vaderscores
0,9.644912e+05,40463.448276,1070.793103,15.241379,84589.429291,3723.437055,97.624292,0.971347,0.971347,82587.974284,0.277400,-0.014347,0.122280,0.046480,0.998404,-0.042517,0.069875,-0.045000,-0.159125,0.099091,0.093435,0.096708
1,1.045160e+06,44341.740000,1192.020000,17.400000,63909.437389,2967.019734,89.903678,0.969618,0.969618,62169.305249,0.319667,-0.081008,0.121952,0.042571,0.998157,0.110200,0.125850,0.059200,-0.075550,0.099556,0.099421,0.100650
2,1.077168e+06,46144.080000,1244.080000,18.640000,64870.463707,3002.939559,91.268385,0.967561,0.967561,63097.778622,0.307000,-0.004042,0.120857,0.039333,0.998038,0.269840,0.166950,0.157000,-0.018450,0.096176,0.097526,0.098400
3,1.077747e+06,46178.200000,1245.080000,18.680000,65211.201800,3031.484044,92.005868,0.969667,0.969667,63429.337457,0.319667,-0.082817,0.121143,0.042048,0.998081,0.190160,0.090150,0.092750,-0.072100,0.096667,0.098474,0.098000
4,1.080107e+06,46259.160000,1248.360000,18.960000,65479.014739,3024.847456,92.011051,0.969174,0.969174,63687.150884,0.319667,-0.022800,0.118905,0.041810,0.997852,0.190000,0.126750,0.101050,-0.098250,0.096176,0.095842,0.097150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,8.561130e+05,44029.640000,966.760000,122.960000,13053.192061,783.768560,19.098946,0.958856,0.958856,12706.300806,0.337000,-0.228971,0.128150,0.041150,0.998830,-0.266840,0.070750,0.196550,0.148350,0.107000,0.116300,0.119550
794,8.570901e+05,44060.440000,968.200000,123.240000,12421.798377,741.488729,16.643729,0.958569,0.958569,12114.157836,0.337000,-0.228971,0.128150,0.041150,0.998830,-0.266840,0.070750,0.196550,0.148350,0.107000,0.116300,0.119550
795,1.007233e+06,50007.560000,1136.760000,131.080000,13108.659614,769.382933,17.351029,0.959595,0.959595,12782.911467,0.362333,-0.132087,0.129850,0.041050,0.999175,-0.186920,0.086800,0.200750,0.131700,0.110579,0.114200,0.117800
796,8.577154e+05,44094.920000,969.280000,123.280000,12679.298844,765.462176,17.285382,0.959735,0.959735,12362.124996,0.337000,-0.241762,0.130600,0.040850,0.999190,-0.266760,0.066500,0.202500,0.126650,0.108632,0.117400,0.119650


In [76]:
X.columns

Index(['viewCount', 'likeCount', 'dislikeCount', 'num_days_since_pub',
       'viewcounts_weighted', 'like_count_weighted', 'dislike_count_weighted',
       'like_dislike_ratio', 'weighted_like_dislike_ratio',
       'like_dislike_ratio_by_viewcounts_weighted',
       'videoTitleSentimentScore_vader_pos',
       'videoTitleSentimentScore_vader_com', 'captionSentimentScore_vader_pos',
       'captionSentimentScore_vader_neg', 'captionSentimentScore_vader_com',
       'falir_videoTitle_score',
       'caption_list_split_by_nvidia_10words_flairscores',
       'caption_list_split_by_nvidia_20words_flairscores',
       'caption_list_split_by_nvidia_35words_flairscores',
       'caption_list_split_by_nvidia_10words_vaderscores',
       'caption_list_split_by_nvidia_20words_vaderscores',
       'caption_list_split_by_nvidia_35words_vaderscores'],
      dtype='object')

In [77]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,

In [67]:
# split x and y
X = final_df.drop(
    [
        "Unnamed: 0",
        "msft",
        "amd",
        "ndaq",
        "intel",
        "qualcomm",
        "apple",
        "pred_24",
        "pred_48",
        "pred_72",
        "action_96",
        "percent_change_24",
        "percent_change_48",
        "percent_change_72",
    ],
    axis=1,
)
y = final_df.iloc[:, -1]

# create lables
le = LabelEncoder()
le.fit(y)
y = le.transform(y)
print(list(le.classes_))
print(le.transform(["buy", "hold", "sell"]))
# Scaling using the Standard Scaler
sc_1 = StandardScaler()
sc_1.fit(X)
X_1 = pd.DataFrame(sc_1.fit_transform(X))

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(
    X_1, y, test_size=0.3, random_state=0
)

['buy', 'hold', 'sell']
[0 1 2]


In [70]:
import lightgbm as lgb
from lightgbm import LGBMModel, LGBMClassifier


model = lgb.LGBMClassifier(
    objective="multiclass", metric="multi_logloss"
)  # ,device="gpu", gpu_platform_id = 0, gpu_device_id = 0)
param_grid = {
    "boosting_type": ["gbdt"],  # , 'goss', 'dart'],
    "num_leaves": [15, 18],  # list(range(30, 150)),
    "learning_rate": list(
        np.logspace(np.log(0.001), np.log(0.05), base=np.exp(1), num=5)
    ),  # 1000
    "n_estimators": list(range(1900, 1910, 5)),  # [1800, 1900, 1950, 2000]
    "bagging_fraction": [0.5, 0.8],
    "bagging_freq": [5, 10]
    # "lgg_grid" : dict(class_weight=class_weight, boosting_type=boosting_type, num_leaves=num_leaves, learning_rate =learning_rate)
}

model, pred = algorithm_pipeline(
    X_train, X_test, y_train, y_test, model, param_grid, cv=5
)

print(model.best_score_)
print(model.best_params_)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  5.9min finished


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
0.8476833976833976
{'bagging_fraction': 0.8, 'bagging_freq': 10, 'boosting_type': 'gbdt', 'learning_rate': 0.018803015465431974, 'n_estimators': 1905, 'num_leaves': 18}


In [71]:
pram_best = model.best_params_

In [72]:
Lgb = lgb.LGBMClassifier(objective="multiclass", metric="multi_logloss")
Lgb.set_params(**pram_best)
fit_model = Lgb.fit(X_train, y_train)
y_test_pred_prob = fit_model.predict_proba(X_test)
y_test_pred = fit_model.predict(X_test)
# print(y_test_pred)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


In [73]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_test_pred)

array([[ 45,  11,   0],
       [  5, 114,  11],
       [  1,  13,  40]], dtype=int64)

In [ ]:
# split x and y
X = final_df.iloc[:, 1:-2]

# create lables
le = LabelEncoder()
le.fit(y)
y = le.transform(y)
print(list(le.classes_))

# Scaling using the Standard Scaler
sc_1 = StandardScaler()
sc_1.fit(X)
X_1 = pd.DataFrame(sc_1.fit_transform(X))

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(
    X_1, y, test_size=0.3, random_state=0
)

# Converting the dataset in proper LGB format
d_train = lgb.Dataset(X_train, label=y_train)

# setting up the parameters
params = {}
params["learning_rate"] = 0.03
params["boosting_type"] = "gbdt"  # GradientBoostingDecisionTree
params["objective"] = "multiclass"  # Multi-class target feature
# params["metric"] = "multi_logloss"  # metric for multi-class
# params["max_depth"] = 10
params[
    "num_class"
] = 3  # no.of unique values in the target class not inclusive of the end value

# training the model
clf = lgb.train(params, d_train, 100)  # training the model on 100 epocs

# prediction on the test dataset
y_pred_1 = clf.predict(X_test)

# printing the predictions
#y_pred_1